# Elasticsearch Queries


## Setup


In [1]:
from datetime import datetime, timezone
from elasticsearch import Elasticsearch
import sys
sys.path.append('..')
import os
os.chdir('../')
from connectors.elk import Elk
elk = Elk()
es:Elasticsearch = elk.es

Elasticsearch: Connection sucessful


# Queries (search)

## BORKED Users not connected from more than X days (ms_signins)


In [3]:
from datetime import datetime, timedelta

# Definición de la consulta con top_hits para obtener detalles adicionales
query = {
    "size": 0,
    "aggs": {
        "users": {
            "terms": {
                "field": "userId.keyword",
                "size": 10000  # Ajusta este número según la cantidad esperada de usuarios
            },
            "aggs": {
                "latest_login": {
                    "max": {
                        "field": "@timestamp"
                    }
                },
                "top_user_info": {
                    "top_hits": {
                        "sort": [
                            {
                                "@timestamp": {
                                    "order": "desc"
                                }
                            }
                        ],
                        "_source": {
                            "includes": ["userDisplayName", "deviceDetail.displayName"]
                        },
                        "size": 1
                    }
                }
            }
        }
    }
}

# Realiza la consulta
# Asegúrate de cambiarlo por el nombre real de tu índice
new_index_name = "logs-ms_singins"
result = es.search(index=new_index_name, body=query)

# Extrae y ordena los resultados por la fecha de última conexión de manera ascendente
users_last_login = []
for bucket in result['aggregations']['users']['buckets']:
    user_id = bucket['key']
    last_login_timestamp = bucket['latest_login']['value_as_string']
    user_info = bucket['top_user_info']['hits']['hits'][0]['_source']
    user_display_name = user_info.get('userDisplayName', 'N/A')
    device_display_name = user_info.get(
        'deviceDetail', {}).get('displayName', 'N/A')
    users_last_login.append(
        (user_id, user_display_name, device_display_name, last_login_timestamp))

# Ordena por la fecha de última conexión de manera ascendente
users_last_login_sorted = sorted(users_last_login, key=lambda x: x[3])

# Define el formato de fecha utilizado en tus timestamps
fecha_formato = "%Y-%m-%dT%H:%M:%S.%fZ"

# Calcula la fecha límite para los usuarios inactivos (hace más de 30 días)
limite_inactividad = datetime.now() - timedelta(days=30)

# Filtra los usuarios inactivos por más de 30 días
usuarios_inactivos = []
for user in users_last_login_sorted:
    user_id, user_display_name, device_display_name, last_login_str = user
    last_login = datetime.strptime(last_login_str, fecha_formato)

    if last_login < limite_inactividad:
        usuarios_inactivos.append(user)

# Imprime los usuarios inactivos
for user in usuarios_inactivos:
    print(user)

('77d6c11b-92a4-45c6-be2e-e6043f11ff00', 'Cabral, Ivonne', '', '2024-01-25T14:02:57.000Z')
('90042398-f05b-4405-8690-f695079ff3e4', 'tdg.galera1', '', '2024-01-25T14:22:30.000Z')
('517439f8-c726-4f42-a251-3f7de24b1015', 'Gutierres Tovar, Victor', '', '2024-01-26T09:34:00.000Z')


## Last connection from a user


In [ ]:
# Define el userId para el cual quieres obtener la última conexión
user_id = "21bf1acf-0ebe-420e-b385-9d4014a11406"

# Construye la consulta
query = {
    "query": {
        "match": {
            "userId": user_id
        }
    },
    "sort": [
        {
            "@timestamp": {
                "order": "desc"
            }
        }
    ],
    "size": 1
}

# Realiza la consulta al índice deseado
new_index_name = "logs-ms_singins"
response = es.search(index=new_index_name, body=query)


print(json.dumps(dict(response), indent=4))

# Extrae y muestra la información relevante de la respuesta
if response["hits"]["hits"]:
    last_login = response["hits"]["hits"][0]["_source"]
    print(f"Última conexión de {last_login['userDisplayName']} ({last_login['userPrincipalName']}):")
    print(f"Fecha y hora: {last_login['@timestamp']}")
    print(f"IP: {last_login['ipAddress']}")
    print(f"Dispositivo: {last_login['deviceDetail']['displayName']}, {last_login['deviceDetail']['operatingSystem']}")
else:
    print("No se encontraron conexiones para el usuario especificado.")

## Users not connected over X days

In [11]:
index = "ms_users"

query = {
    "query": {
        "bool": {
            "filter": [
                {
                    "range": {
                        "last_signin": {
                            "lt": "now-1M/M"
                        }
                    }
                },
                {
                    "term": {
                        "accountEnabled": {
                            "value": True
                        }
                    }
                }
            ],
            "must": [
                {
                    "terms": {
                        "assignedLicenses.name.keyword": ["E3", "threat_protection"]
                    }
                }
            ]

        }
    },
    "size": 100
}


response = es.search(index=index, body=query)
print(response["hits"]["total"]["value"])

for hit in response['hits']['hits']:
    # Assuming the user's name or ID is stored under a field named 'displayName' or '_id'
    user_name = hit['_source'].get('displayName', 'No Name')
    user_id = hit['_id']
    print(f'User ID: {user_id}, Name: {user_name}')

23
User ID: 3ba25803-a027-493a-8445-2d13685dcae2, Name: Atencion cliente Tabacalera
User ID: 690ea146-1ebe-48b0-a361-5a3a328d3c1d, Name: Comunicaciones FDC
User ID: f2fe093b-f201-4ade-a92c-9e59d292cc75, Name: Digitador Tabaco
User ID: d8e85e22-f4dd-4441-8754-97400e8fae35, Name: Premium connection
User ID: c9a74f98-b5fb-46bb-9b60-593c72005745, Name: Comunicación Tabacalera
User ID: a356ba03-dc21-46c9-82ee-08dc02b38210, Name: FDC Test User
User ID: 0c63a527-3d9c-4534-865d-d672ae25e5ce, Name: Ramirez, Francisca
User ID: 7a5cc711-053e-466b-9949-e8d37ca8edcc, Name: HRPortal-noreply
User ID: 9573c9fe-e056-48fc-9667-e9b6f0029554, Name: Impresora_FDC
User ID: a3698c4e-1397-4f98-8046-452a8a42b28c, Name: Aguilar, Jose Edgardo
User ID: e807cde1-6310-4e39-96b2-93c47059c062, Name: Yrrizarri, Lorenzo
User ID: 23277739-ad97-427e-9f9c-2d29fb436998, Name: Polycom
User ID: f6423643-7bc1-47ce-8e55-dc5f58f38d41, Name: Registros Promocigar
User ID: 6662cef8-0603-46b5-9668-9f06da5f4593, Name: RRHH Externo
U

# Search engines (search multimatch)

## User searcher


In [ ]:
response = es.search(
    index="ms_users",
    query={
        "multi_match": {
            "query":    "victor gutierrez",
            "fields": ["displayName", "deviceDisplayName"]
        }
    }
)

print(json.dumps(dict(response), indent=2))
print(json.dumps(response["hits"]["hits"][0]["_source"]["mail"], indent=2))

# Reindex

In [7]:
new_index_name = "ms_users"
old_index_name = "ms_users2"

reindex_body = {
    "source": {"index": old_index_name},
    "dest": {"index": new_index_name}
}
es.reindex(body=reindex_body)

print(f"Datos reindexados de {old_index_name} a {new_index_name}.")

Datos reindexados de ms_users2 a ms_users.


## With a filter

In [3]:
new_index_name = "logs-ms_signins_interactive"
old_index_name = "logs-ms_singins"

field_key = "isInteractive"
field_value = True

reindex_body = {
  "source": {
    "index": old_index_name,
    "query": {
      "match": {
        field_key: field_value
      }
    }
  },
  "dest": {
    "index": new_index_name
  }
}

es.reindex(body=reindex_body, wait_for_completion=False)

print(f"Los datos se estan reindexando de {old_index_name} a {new_index_name}.")

Los datos se estan reindexando de logs-ms_singins a logs-ms_signins_interactive.


# Delete a field of a index

In [3]:
index = "ms_users"
field_key = "last_singin"
# The script to remove the field from all documents
script = {
    "script": {
        "source": f"ctx._source.remove('{field_key}')",
        "lang": "painless"
    },
    "query": {
        "exists": {
            "field": field_key
        }
    }
}

# Execute the Update By Query API to delete the field
response = es.update_by_query(index=index, body=script, refresh=True)

# Print the response
print(response)

{'took': 5, 'timed_out': False, 'total': 0, 'updated': 0, 'deleted': 0, 'batches': 0, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
